In [ ]:
# from drn_interactions.stats import cv_isi_burst, mannwhitneyu_plusplus
from drn_interactions.load import load_derived_generic, load_neurons_derived, get_eeg_sessions
from drn_interactions.spikes import SpikesHandler
from drn_interactions.surrogates import shuffle_spikes
from drn_interactions.brain_state import get_state_piv, get_state_long
from drn_interactions.brain_state_transforms import BrainStateUtils
from drn_interactions.brain_state import BSResonders
from scipy.stats import zscore
from drn_interactions.transforms import segment_spikes
from drn_interactions.stats import cv_isi_burst

import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import numpy as np
import pandas as pd
sns.set_theme(context="poster", style="ticks")

from IPython.display import display


%load_ext autoreload
%autoreload 2

In [ ]:
eeg_states = load_derived_generic("eeg_states.csv").rename(columns={"cluster": "state"})
neurons = load_neurons_derived()
sessions = get_eeg_sessions().tolist()
clusters = neurons[["neuron_id", "wf_3"]]
sh = SpikesHandler(
    "pre", 
    bin_width=1, 
    session_names=sessions,
     t_start=0, 
     t_stop=1800
    )

In [ ]:
transformer = BrainStateUtils()

df = transformer.align_spikes_state_long(sh, eeg_states)
df["counts"] = df.groupby("neuron_id")["counts"].transform(zscore)
df.head(3)

In [ ]:
# spike rate

mod = BSResonders(within_first=False)
out = mod.get_anova(df)
display(out["anova"])
display(out["contrasts"])

# with neuron types
out = mod.get_anova(df, clusters=clusters)
display(out["anova"])
display(out["contrasts"])

In [ ]:
responders = mod.get_responders(df, abs_diff_thresh=0.2)
mod.plot_responders(clusters=clusters, bins=np.arange(-2.5, 2.5, 0.2))

In [ ]:
# cv isi


def get_cvisi_in_segment(sh, segment_length=120):
    spikes = sh.spikes.copy()
    spikes = segment_spikes(
        spikes, t_start=sh.t_start, t_stop=sh.t_stop, segment_length=segment_length
    )
    return (
        spikes.groupby(["neuron_id", "segment"])["spiketimes"]
        .apply(cv_isi_burst)
        .to_frame("cv")
        .reset_index()
        .dropna()
    )



spikes = get_cvisi_in_segment(sh, segment_length=60)
df = transformer.align_segmentedspikes_state(spikes, eeg_states, )

In [ ]:
mod = BSResonders(df_value_col="cv", within_first=False)
out = mod.get_anova(df)
display(out["anova"])
display(out["contrasts"])

# with neuron types
out = mod.get_anova(df, clusters=clusters)
display(out["anova"])
display(out["contrasts"])

In [ ]:
responders = mod.get_responders(df, abs_diff_thresh=0.01)
# mod.plot_responders(clusters=clusters,
# bins="auto",
# #  bins=np.arange(-2.5, 2.5, 0.2)
#  )